## 데이터가 커서 유저별로 전처리 실행 후 merge
### 개인 데이터여서 해당 데이터 파일은 올리지 않음

In [ ]:
df= pd.read_csv("u1.csv")

In [ ]:
df = df[df['notification_id'].notna()]
df = df.dropna(axis=1, how='all')
df

In [ ]:
df = df[df['notification_id'].notna()]
df = df.dropna(axis=1, how='all')
df

In [ ]:
condition = df['post_time'].notna() & (df['text'] == 'No Text') & (df['title'] == 'No Title')
df = df[~condition]
df

In [ ]:
no_text_title = (df['text'] == 'No Text') & (df['title'] == 'No Title')

df['removal_reason2'] = pd.NA

def propagate_removal_reason(row):
    if pd.notna(row['removal_time']) and pd.notna(row['package_name']):
        condition = (df['removal_time'] == row['removal_time']) & \
                    (df['package_name'] == row['package_name']) & \
                    no_text_title
        reasons = df.loc[condition, 'removal_reason'].dropna().unique()
        if reasons.size > 0:
            return reasons[0]
    return row['removal_reason2']

df['removal_reason2'] = df.apply(propagate_removal_reason, axis=1)

df[['removal_reason2', 'removal_time', 'package_name']].head()

In [ ]:
# 제거된 알림 중에 text, title 둘다 없는 데이터 제거
remove_condition = no_text_title & df['removal_reason'].notna()
df = df[~remove_condition]
# 값이 other이라면 제거 
df.loc[df['removal_reason2'] == 'Other', 'removal_reason2'] = pd.NA
# removal_reason 값이 Group Summary Canceled라면, removal_reason2 값을 대체
df.loc[df['removal_reason'] == 'Group Summary Canceled', 'removal_reason'] = df['removal_reason2']
# removal_reason과 removal_reason2의 값이 같다면 removal_reason2 값 제거
df.loc[df['removal_reason'] == df['removal_reason2'], 'removal_reason2'] = pd.NA
df

In [ ]:
filtered_data = df[df['removal_reason2'].notnull()]
filtered_data.info()

In [ ]:
df = df.drop(columns=['removal_reason2'])

In [ ]:
# 1. timestamp, post_time 및 removal_time 열을 datetime 형식으로 변환
df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')
df['post_time'] = pd.to_datetime(df['post_time'], errors='coerce')
df['removal_time'] = pd.to_datetime(df['removal_time'], errors='coerce')

# 2. timestamp 열을 기준으로 데이터프레임을 시간순으로 정렬
df = df.sort_values(by='timestamp').reset_index(drop=True)
# 설정 알림 제외
df =  df[~((df['notification_id'] == 1.0) & (df['package_name'] == 'com.example.uxchannel_proto'))]
df =  df[~((df['notification_id'] == 2.0) & (df['package_name'] == 'com.example.uxchannel_proto'))]

In [ ]:
df = df[(df['removal_time'].isna()) | (~df['removal_reason'].isna())]
df

In [ ]:
# 3. 모든 행을 포함하도록 df_with_removal을 생성합니다.
df_with_removal = df.copy()

# 4. removal_time이 있는 행들에 대해 처리합니다.
results = []
for index, row in df_with_removal[df_with_removal['removal_time'].notnull()].iterrows():
    # 동일한 deviceId, notification_id, package_name, text, title을 가진 행들을 찾습니다.
    condition = (
        (df_with_removal['deviceId'] == row['deviceId']) &
        (df_with_removal['notification_id'] == row['notification_id']) &
        (df_with_removal['package_name'] == row['package_name']) &
        (df_with_removal['text'] == row['text']) &
        (df_with_removal['title'] == row['title']) &
        (df_with_removal['post_time'] < row['removal_time'])
    )
    
    # 조건을 만족하는 가장 최신의 post_time을 찾습니다.
    latest_post_row = df_with_removal[condition].sort_values(by='post_time').tail(1)
    
    # 결과 리스트에 추가합니다.
    if not latest_post_row.empty:
        results.append(latest_post_row)
    
    # 현재 row(즉, removal_time이 있는 row)도 결과 리스트에 추가합니다.
    results.append(df_with_removal.loc[[index]])

# 5. 결과 리스트를 데이터프레임으로 병합합니다.
df_final = pd.concat(results).drop_duplicates().reset_index(drop=True)

In [ ]:
# 필요없는 컬럼 제거 및 시간 변환
df_final['post_time'] = pd.to_datetime(df_final['post_time'], errors='coerce')
df_final['removal_time'] = pd.to_datetime(df_final['removal_time'], errors='coerce')
# post_time과 removal_time을 각각 가지고 있는 데이터프레임 생성
posts = df_final.dropna(subset=['post_time']).drop(columns=['removal_time', 'removal_reason'])
removals = df_final.dropna(subset=['removal_time']).drop(columns=['post_time'])

In [ ]:
# 게시 데이터와 제거 데이터를 병합하여 알림 매칭
merged_data = pd.merge(removals, posts, on=['deviceId', 'notification_id', 'package_name', 'text', 'title'], how='left')

# post_time이 removal_time 이전인 데이터만 필터링
merged_data = merged_data[merged_data['post_time'] < merged_data['removal_time']]

# 각 removal_time에 가장 가까운 최신 post_time 선택
merged_data['post_time'] = merged_data.groupby(['deviceId', 'notification_id', 'package_name', 'text', 'title', 'removal_time'])['post_time'].transform(max)

In [ ]:
df_noti_final = merged_data.drop_duplicates(subset=['deviceId', 'notification_id', 'package_name', 'text', 'title', 'removal_time'])
df_noti_final

In [ ]:
df_noti_final['time_difference'] = df_noti_final['removal_time'] - df_noti_final['post_time']
df_noti_final['time_difference_hms'] = df_noti_final['time_difference'].apply(lambda x: f"{int(x.total_seconds() // 3600)}:{int((x.total_seconds() % 3600) // 60)}:{int(x.total_seconds() % 60)}")
df_noti_final

In [ ]:
df_noti_final.to_csv('u1_noti.csv', index=False)

## 파일 병합

In [ ]:
import pandas as pd
import os

# Get all CSV files in the current folder ending with '_noti.csv'
files = [f for f in os.listdir() if f.endswith("_noti.csv")]

# Combine all files into one dataframe
combined_df = pd.DataFrame()

for file in files:
    df = pd.read_csv(file)  # Read each CSV file
    combined_df = pd.concat([combined_df, df], ignore_index=True)  # Append data

# Save the combined dataframe to a new CSV file
output_file = "combined_noti.csv"
combined_df.to_csv(output_file, index=False)

print(f"Combined file saved as {output_file}")

In [ ]:
df = df_noti_final.drop(columns=['time_difference_hms'])

In [ ]:
df['contains_advertisement'] = df['text'].str.contains('광고', case=False, na=False) | \
                                 df['title'].str.contains('광고', case=False, na=False)

In [ ]:
df.loc[df['removal_reason'] == 'Notification Swiped', 'removal_reason'] = '제거'
df.loc[df['removal_reason'] == 'All Notifications Cleared', 'removal_reason'] = '제거'
df.loc[df['removal_reason'] == 'Notification Clicked', 'removal_reason'] = '클릭'
df.loc[df['removal_reason'] == 'App Specific Cancel', 'removal_reason'] = '기타'
df.loc[df['removal_reason'] == 'Channel Removed', 'removal_reason'] = '기타'
df.loc[df['removal_reason'] == 'Other', 'removal_reason'] = '기타'
df.loc[df['removal_reason'] == 'Timeout', 'removal_reason'] = '기타'
df.loc[df['removal_reason'] == 'Package Changed', 'removal_reason'] = '기타'
df.loc[df['removal_reason'] == 'App Cancel All Notifications', 'removal_reason'] = '기타'
df.loc[df['removal_reason'] == 'Package Banned', 'removal_reason'] = '기타'
df.loc[df['removal_reason'] == 'Lockdown', 'removal_reason'] = '기타'
df.loc[df['removal_reason'] == 'Listener Cancel', 'removal_reason'] = '중요도 설정'

In [ ]:
package_removal_counts = df.groupby(['package_name', 'removal_reason']).size().unstack(fill_value=0)
package_row_counts = df['package_name'].value_counts()
package_removal_counts['total_count'] = package_row_counts
package_removal_counts